# **Assignment 4**

# **1.Load the dataset**

In [12]:
dataset_location = "/content/drive/MyDrive/spam.csv"

# **2.Import the Library**

In [13]:
import pandas as pd
import nltk
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.translate.ribes_score import word_rank_alignment
from numpy.lib.shape_base import split
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import LSTM,Dense,Dropout,Input,Embedding,Activation,Flatten
from keras.models import Model
import nltk

# **3. Read dataset and do preprocessing**

In [14]:
data = pd.read_csv(dataset_location,encoding = "ISO-8859-1")

In [15]:
data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis = 1,inplace = True)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [19]:
nltk.download('stopwords',quiet=True)
nltk.download('all',quiet=True)

True

In [20]:
ps = PorterStemmer()
input = []

In [21]:
for i in range(0,5572):
  v2 = data['v2'][i]

  #removing punctuation
  v2 = re.sub('[^a-zA-Z]',' ',v2)

  #converting to lower case
  v2 = v2.lower()

  #splitting the sentence
  v2 = v2.split()

  #removing the stopwords and stemming
  v2 = [ps.stem(word) for word in v2 if not word in set(stopwords.words('english'))]

  v2 = ' '.join(v2)

  input.append(v2)

In [22]:
#creating document term matrix
cv = CountVectorizer(max_features=2000)
x = cv.fit_transform(input).toarray()
x.shape

(5572, 2000)

In [23]:
le = preprocessing.LabelEncoder()

data['v1'] = le.fit_transform(data['v1'])
data['v1'].unique()

array([0, 1])

In [24]:
y = data['v1'].values

In [25]:
y = y.reshape(-1,1)

In [26]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.4)

# **4. Model building - Adding layers, Compiling model and saving model**

In [27]:
model = Sequential()

In [28]:
model.add(Dense(1565,activation = "relu"))
model.add(Dense(3000,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))
model.add(Flatten())

In [29]:
model.compile(optimizer = "adam",loss = "binary_crossentropy", metrics = ["accuracy"])

In [30]:
model.fit(x_train,y_train,epochs = 15)

Epoch 1/15
105/105 [==============================] - 9s 73ms/step - loss: 0.1485 - accuracy: 0.9581
Epoch 2/15
105/105 [==============================] - 7s 70ms/step - loss: 0.0132 - accuracy: 0.9961
Epoch 3/15
105/105 [==============================] - 10s 91ms/step - loss: 0.0049 - accuracy: 0.9982
Epoch 4/15
105/105 [==============================] - 7s 71ms/step - loss: 0.0031 - accuracy: 0.9994
Epoch 5/15
105/105 [==============================] - 7s 70ms/step - loss: 0.0028 - accuracy: 0.9994
Epoch 6/15
105/105 [==============================] - 7s 70ms/step - loss: 0.0028 - accuracy: 0.9991
Epoch 7/15
105/105 [==============================] - 7s 70ms/step - loss: 0.0027 - accuracy: 0.9991
Epoch 8/15
105/105 [==============================] - 7s 70ms/step - loss: 0.0025 - accuracy: 0.9991
Epoch 9/15
105/105 [==============================] - 7s 71ms/step - loss: 0.0024 - accuracy: 0.9994
Epoch 10/15
105/105 [==============================] - 7s 70ms/step - loss: 0.0024 - accur

In [31]:
model.save("spam-message-classifier.h5")

# **5. Testing the model**

In [32]:
ham = "im donee. come pick me up"
spam = "WINNER$$$$ SMS REPLY 'WIN'"
message = re.sub('[^a-zA-Z]',' ',spam)
message

'WINNER     SMS REPLY  WIN '

# **Testing with spam message**

In [33]:
message = message.split()
message = [ps.stem(word) for word in message if not word in set(stopwords.words('english')) ]
message = ' '.join(message)

In [34]:
message1 = cv.transform([message])
message1

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [35]:
TruePredction = model.predict(message1.astype(float))

1/1 [==============================] - 0s 160ms/step


In [36]:
TruePredction > 0.5

array([[ True]])

# **Testing with normal message**

In [37]:
msg = re.sub('[^a-zA-Z]',' ',ham)
msg

'im donee  come pick me up'

In [38]:
msg = msg.split()
msg = [ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]
msg = ' '.join(msg)

In [39]:
msg

'im done come pick'

In [40]:
cv.transform([msg])

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [41]:
FalsePredection = model.predict(cv.transform([msg]))

1/1 [==============================] - 0s 94ms/step


In [42]:
FalsePredection > 0.5

array([[False]])